### Import Statements:

In [9]:
import urllib
import os
import brotli
import csv
import html
import time
import random
import json
import requests
from pathlib import PurePath
from collections import defaultdict
import pandas as pd

### Defining Functions:

In [10]:
def wait_random_time(min_time, max_time):
    """Waits for a random amount of time between min_time and max_time (inclusive)."""
    sleep_time = random.uniform(min_time, max_time)
    print(f"Waiting random amount of seconds ({round(sleep_time,2)}s in this case) before continuing...")
    time.sleep(sleep_time)

In [11]:
def progress_bar(list_length, index):
    if list_length == 96:         # Most times, each page of https://pitchfork.com/reviews/albums/?page=x
                                  # has 96 albums...
        if index == 12:
            print("~\n█░░░░░░░ 12.5%")
        elif index == 24:
            print("~\n███░░░░░░░░░ 25%")
        elif index == 36:
            print("~\n████░░░░░░░ 37%.5")
        elif index == 48:
            print("~\n██████░░░░░░ 50%")
        elif index == 60:
            print("~\n████████░░░░░ 62.5%")
        elif index == 72:
            print("~\n█████████░░░ 75%")
        elif index == 84:
            print("~\n█████████▓░ 87.5%")
            
    if list_length == 190:      # However, one request I made showed 190 albums per page. This might have been
                                # a fluke (it was around the time P4k added new reviews to their site... I also
                                # got a 502 Bad Gateway error around this time)
        if index == 19:
            print("~\n█░░░░░░░░░ 10%")
        elif index == 38:
            print("~\n██░░░░░░░░ 20%")
        elif index == 57:
            print("~\n███░░░░░░░ 30%")
        elif index == 76:
            print("~\n████░░░░░░ 40%")
        elif index == 95:
            print("~\n██████░░░░░░ 50%")
        elif index == 114:
            print("~\n██████░░░░ 60%")
        elif index == 133:
            print("~\n███████░░░ 70%")
        elif index == 152:
            print("~\n	████████░░ 80%")
        elif index == 171:
            print("~\n█████████░ 90%")

In [12]:
def RequestAlbumsPage(page_number, song_dict_):
    looper = "on"
    while looper == "on":
        try:
            wait_random_time(9,33)
            pitchfork_pg_x = r'https://pitchfork.com/reviews/albums/?page=' + f'{page_number}'
            print(f"\nRequesting page {page_number} of pitchfork.com/reviews/albums ...\n")
            req = urllib.request.Request(pitchfork_pg_x) # Constructing HTTP request obj
            with urllib.request.urlopen(req) as page_response:
                http_status_code = page_response.code
                r = page_response.read()
                bebe_decoded = r.decode("utf-8")
    
                # list split up by album text code things:
                splitbythings = bebe_decoded.split('"@type":"ListItem","name":')

                clean_split_items = []

                for i in range(len(splitbythings)):
                    if i == 0:
                        pass
                    else:
                        itersplit_dirty = splitbythings[i]
                        itersplit_1 = itersplit_dirty.split(',"position"')[0]
                        if r"\u002Freviews\u002Falbums" not in itersplit_1:
                            clean_split_items.append([itersplit_1, itersplit_dirty])

                for item in clean_split_items:
                    clean = item[0]
                    dirty = item[1]
                    firstsplit = clean.split('"*')[1]
                    albumname_rough_parse = firstsplit.split('","url":"')[0]
                    secondsplit = firstsplit.split('","url":"')[1]
                    url_parsed = secondsplit.split('/"')[0]

                    if albumname_rough_parse[-1:] == '*':
                         albumname_clean = albumname_rough_parse[:-1]
                    elif albumname_rough_parse[-1:] == 'P':
                        albumname_clean = albumname_rough_parse[:-4] + " <EP>"
                    else:
                        print("Woah buddy, you haven't accounted for this case!")

                    url_purepath = PurePath(url_parsed)
                    url_songdesc = url_purepath.name

                    song_dict_[url_songdesc]['album_name'] = albumname_clean
                    song_dict_[url_songdesc]['review_url'] = url_parsed


                    song_split_str = url_songdesc + '"},"showAssetOnly"'
                    artistname_split_1 = itersplit_dirty.split(song_split_str)
                    artistname_split_2 = artistname_split_1[0].split('"subHed":{"name":"')[-1]
                    artist_name_parsed = artistname_split_2.split('","url":')[0]

                    song_dict_[url_songdesc]['artist_name'] = artist_name_parsed
                    
                looper = "off"
                break
                    
        except urllib.error.HTTPError as e:
            # Handle specific non-200 HTTP errors
            print(f"\n!!!!! HTTP Error: Status code {e.code}. Retrying...\n")
        except urllib.error.URLError as e:
            # Handle other URL errors (e.g., connection issues, invalid URL)
            print(f"\n!!!!! URL Error: {e.reason}. Retrying...\n")
        except Exception as e:
            # Handle any other unexpected exceptions
            print(f"\n!!!!! An unexpected error occurred: {e}. Retrying...\n")
            
            
            
def RequestIndividualReviewPage(page_number, song_dict_):
    song_dict_reg = dict(song_dict_)
    for index, key_string in enumerate(song_dict_reg):
        indiv_looper = "on"
        progress_bar(len(song_dict_reg), index)
        while indiv_looper == "on":
            try:
                wait_random_time(9,33)
                #progress_bar(len(song_dict_reg), index)

                individual_review_url = song_dict_reg[key_string]['review_url']
                print(f"~\nPrepping to request {key_string} ({index+1}/{len(song_dict_reg)} - page {page_number})...")
                req = urllib.request.Request(individual_review_url)
                r = urllib.request.urlopen(req).read()
                jeen_decoded = r.decode("utf-8")

                infoslicefields_split1 = jeen_decoded.split(',"infoSliceFields":')[1]
                infoslicefields_split2 = infoslicefields_split1.split(',"isMusicReview":')
                infoslicefields = json.loads(infoslicefields_split2[0])

                genre_parsed = infoslicefields['genre']
                song_dict_reg[key_string]['genre'] = genre_parsed

                try:
                    label_parsed = infoslicefields['label']
                    song_dict_reg[key_string]['label'] = label_parsed
                except:
                    song_dict_reg[key_string]['label'] = ""

                reviewDate_parsed = infoslicefields['reviewDate']
                song_dict_reg[key_string]['review_date'] = reviewDate_parsed

                releaseYear_parsed = infoslicefields['releaseYear']
                song_dict_reg[key_string]['release_date'] = releaseYear_parsed

                nextpart_split1 = infoslicefields_split1.split('},"isMusicReview":')[1]
                nextpart_split2 = nextpart_split1.split(',"lede":{')[0]
                nextpart_split3 = nextpart_split2.split(',"isTrackReview":')
                isMusicReview_parsed = nextpart_split3[0]
                song_dict_reg[key_string]['is_music_review'] = isMusicReview_parsed

                nextpart_split4 = nextpart_split3[1].split(',"musicRating":')
                isTrackReview_parsed = nextpart_split4[0]
                song_dict_reg[key_string]['is_track_review'] = isTrackReview_parsed

                nextpart_split5 = nextpart_split4[1].split(',"modifiedDate":"')
                lildict = json.loads(nextpart_split5[0])
                isBestNewMusic_parsed = lildict['isBestNewMusic']
                song_dict_reg[key_string]['is_best_new_music'] = isBestNewMusic_parsed

                isBestNewReissue_parsed = lildict['isBestNewReissue']
                song_dict_reg[key_string]['is_best_new_reissue'] = isBestNewReissue_parsed

                score_parsed = lildict['score']
                song_dict_reg[key_string]['rating'] = score_parsed

                authorname_split1 = jeen_decoded.split('><meta name="author" content="')
                authorname_parsed = authorname_split1[1].split('"/><meta name=')[0]
                song_dict_reg[key_string]['review_author_name'] = authorname_parsed

                descrip_split1 = jeen_decoded.split('><meta name="description" content="')[1]
                descrip_parsed = descrip_split1.split('"/><meta name="id" content="')[0]
                song_dict_reg[key_string]['review_description'] = descrip_parsed

                content_id_split1 = jeen_decoded.split('><meta name="id" content="')[1]
                content_id_parsed = content_id_split1.split('"/><meta name="keywords" content="')[0]
                song_dict_reg[key_string]['content_id'] = content_id_parsed

                photo_id_split1 = jeen_decoded.split(r'https://media.pitchfork.com/photos/')[1]
                photo_id_parsed = photo_id_split1.split(r'/16:9/w_1000,c_limit/')[0]
                song_dict_reg[key_string]['photo_id'] = photo_id_parsed

                full_photo_url = r'https://media.pitchfork.com/photos/' + photo_id_parsed + r'/master/w_1280,c_limit'
                song_dict_reg[key_string]['full_photo_url'] = full_photo_url
                
                indiv_looper = "off"
                
            except urllib.error.HTTPError as e:
                # Handle specific non-200 HTTP errors
                print(f"\n!!!!! HTTP Error: Status code {e.code}. Retrying...\n")
            except urllib.error.URLError as e:
                # Handle other URL errors (e.g., connection issues, invalid URL)
                print(f"\n!!!!! URL Error: {e.reason}. Retrying...\n")
            except Exception as e:
                # Handle any other unexpected exceptions
                print(f"\n!!!!! An unexpected error occurred: {e}. Retrying...\n")

    print("██████████ 100% -- COMPLETE")
    return song_dict_reg

# Running the script:

#### Create empty dictionary:

In [13]:
mydicto = {}
mydicto = defaultdict(dict)

#### Enter in page of pitchfork.com/reviews/album you will be scraping:

In [14]:
current_albums_page = input("What album page are you scraping? Type in just the page number integer: ")
print(" ")
RequestAlbumsPage(current_albums_page, mydicto)
myreturn = RequestIndividualReviewPage(current_albums_page, mydicto)

What album page are you scraping? Type in just the page number integer: 7
 
Waiting random amount of seconds (30.66s in this case) before continuing...

Requesting page 7 of pitchfork.com/reviews/albums ...

Waiting random amount of seconds (11.1s in this case) before continuing...
~
Prepping to request yetsuby-4eva (1/96 - page 7)...
Waiting random amount of seconds (27.62s in this case) before continuing...
~
Prepping to request juana-rozas-tanya (2/96 - page 7)...
Waiting random amount of seconds (28.51s in this case) before continuing...
~
Prepping to request momma-welcome-to-my-blue-sky (3/96 - page 7)...
Waiting random amount of seconds (11.59s in this case) before continuing...
~
Prepping to request will-smith-based-on-a-true-story (4/96 - page 7)...
Waiting random amount of seconds (21.64s in this case) before continuing...
~
Prepping to request huremic-seeking-darkness (5/96 - page 7)...
Waiting random amount of seconds (12.67s in this case) before continuing...
~
Prepping to 

~
Prepping to request saba-no-id-from-the-private-collection-of-saba-and-no-id (54/96 - page 7)...
Waiting random amount of seconds (22.9s in this case) before continuing...
~
Prepping to request selena-gomez-benny-blanco-i-said-i-love-you-first (55/96 - page 7)...
Waiting random amount of seconds (28.58s in this case) before continuing...
~
Prepping to request dutch-interior-moneyball (56/96 - page 7)...
Waiting random amount of seconds (32.29s in this case) before continuing...
~
Prepping to request sharp-pins-radio-ddr (57/96 - page 7)...
Waiting random amount of seconds (24.76s in this case) before continuing...
~
Prepping to request more-eaze-claire-rousay-no-floor (58/96 - page 7)...
Waiting random amount of seconds (30.88s in this case) before continuing...
~
Prepping to request lucy-liyou-every-video-without-your-face-every-sound-without-your-name (59/96 - page 7)...
Waiting random amount of seconds (16.53s in this case) before continuing...
~
Prepping to request black-flag-my-

#### Save returned dictionary to a csv (enter in csv name)

In [15]:
df = pd.DataFrame.from_dict(myreturn, orient='index')
df.index.name = 'Album_Key'

datafolder = r'C:\Users\michael.felzan\Desktop\Misc\Pitchforkle\Data'

output_csv = os.path.join(datafolder, f"page{current_albums_page}_12202025.csv") # replace with auto-datetime function
df.to_csv(output_csv)

#### Cycle through returned dictionary of song info, request and save album thumbnails for each review:

In [16]:
datafolder = r'C:\Users\michael.felzan\Desktop\Misc\Pitchforkle\Data\Album_Art'

for index, key_ in enumerate(myreturn):
    pic_looper = "on"
    progress_bar(len(myreturn), index)
    image_url = myreturn[key_]['full_photo_url']
    image_name = key_ + ".png"
    while pic_looper == "on":
        try:
            wait_random_time(9,21)
            img_data = requests.get(image_url).content
            image_output = os.path.join(datafolder, image_name)
            with open(image_output, 'wb') as handler:
                handler.write(img_data)
                print(f"saved {key_}.png ({index+1}/{len(myreturn)})\n~")
                pic_looper = "off"
        except urllib.error.HTTPError as e:
            # Handle specific non-200 HTTP errors
            print(f"\n!!!!! HTTP Error: Status code {e.code}. Retrying...\n")
        except urllib.error.URLError as e:
            # Handle other URL errors (e.g., connection issues, invalid URL)
            print(f"\n!!!!! URL Error: {e.reason}. Retrying...\n")
        except Exception as e:
            # Handle any other unexpected exceptions
            print(f"\n!!!!! An unexpected error occurred: {e}. Retrying...\n")
print("██████████ 100% -- COMPLETE")

Waiting random amount of seconds (15.38s in this case) before continuing...
saved yetsuby-4eva.png (1/96)
~
Waiting random amount of seconds (19.09s in this case) before continuing...
saved juana-rozas-tanya.png (2/96)
~
Waiting random amount of seconds (17.53s in this case) before continuing...
saved momma-welcome-to-my-blue-sky.png (3/96)
~
Waiting random amount of seconds (9.05s in this case) before continuing...
saved will-smith-based-on-a-true-story.png (4/96)
~
Waiting random amount of seconds (9.96s in this case) before continuing...
saved huremic-seeking-darkness.png (5/96)
~
Waiting random amount of seconds (20.95s in this case) before continuing...
saved walt-mcclements-on-a-painted-ocean.png (6/96)
~
Waiting random amount of seconds (12.96s in this case) before continuing...
saved gregory-uhlmann-josh-johnson-sam-wilkes-uhlmann-johnson-wilkes.png (7/96)
~
Waiting random amount of seconds (11.81s in this case) before continuing...
saved duendita-a-strong-desire-to-survive.png

saved jorg-kuning-elvers-pass.png (64/96)
~
Waiting random amount of seconds (15.53s in this case) before continuing...
saved nels-cline-consentrik-quartet.png (65/96)
~
Waiting random amount of seconds (19.63s in this case) before continuing...
saved anoushka-shankar-alam-khan-sarathy-korwar-chapter-iii-we-return-to-light-ep.png (66/96)
~
Waiting random amount of seconds (18.71s in this case) before continuing...
saved ghost-mountain-october-country.png (67/96)
~
Waiting random amount of seconds (10.29s in this case) before continuing...
saved patterson-hood-exploding-trees-and-airplane-screams.png (68/96)
~
Waiting random amount of seconds (9.76s in this case) before continuing...
saved currensy-harry-fraud-never-catch-us.png (69/96)
~
Waiting random amount of seconds (17.53s in this case) before continuing...
saved twin-shadow-georgie.png (70/96)
~
Waiting random amount of seconds (18.69s in this case) before continuing...
saved playboi-carti-music.png (71/96)
~
Waiting random amoun

## To Do:

- write code for combining spreadsheets together
- use datetime package to auto-populate the date portion of the spreadsheet output file names
- 
# make 1% bar
# Combine album image scraping code with review info scraping code


### The following code is for debugging / testing new features!

In [ ]:


individual_review_url = r'https://pitchfork.com/reviews/albums/yungmorpheus-dirty-art-club-a-spyglass-to-ones-face/'
req = urllib.request.Request(individual_review_url)
r = urllib.request.urlopen(req).read()
jeen_decoded = r.decode("utf-8")

infoslicefields_split1 = jeen_decoded.split(',"infoSliceFields":')[1]
infoslicefields_split2 = infoslicefields_split1.split(',"isMusicReview":')
infoslicefields = json.loads(infoslicefields_split2[0])

genre_parsed = infoslicefields['genre']
#song_dict_reg[key_string]['genre'] = genre_parsed

try:
    label_parsed = infoslicefields['label']
    #song_dict_reg[key_string]['label'] = label_parsed
except:
    pass

reviewDate_parsed = infoslicefields['reviewDate']
#song_dict_reg[key_string]['review_date'] = reviewDate_parsed

releaseYear_parsed = infoslicefields['releaseYear']
#song_dict_reg[key_string]['release_date'] = releaseYear_parsed

nextpart_split1 = infoslicefields_split1.split('},"isMusicReview":')[1]
nextpart_split2 = nextpart_split1.split(',"lede":{')[0]
nextpart_split3 = nextpart_split2.split(',"isTrackReview":')
isMusicReview_parsed = nextpart_split3[0]
#song_dict_reg[key_string]['is_music_review'] = isMusicReview_parsed

nextpart_split4 = nextpart_split3[1].split(',"musicRating":')
isTrackReview_parsed = nextpart_split4[0]
#song_dict_reg[key_string]['is_track_review'] = isTrackReview_parsed

nextpart_split5 = nextpart_split4[1].split(',"modifiedDate":"')
lildict = json.loads(nextpart_split5[0])
isBestNewMusic_parsed = lildict['isBestNewMusic']
#song_dict_reg[key_string]['is_best_new_music'] = isBestNewMusic_parsed

isBestNewReissue_parsed = lildict['isBestNewReissue']
#song_dict_reg[key_string]['is_best_new_reissue'] = isBestNewReissue_parsed

score_parsed = lildict['score']
#song_dict_reg[key_string]['rating'] = score_parsed

authorname_split1 = jeen_decoded.split('><meta name="author" content="')
authorname_parsed = authorname_split1[1].split('"/><meta name=')[0]
#song_dict_reg[key_string]['review_author_name'] = authorname_parsed

descrip_split1 = jeen_decoded.split('><meta name="description" content="')[1]
descrip_parsed = descrip_split1.split('"/><meta name="id" content="')[0]
#song_dict_reg[key_string]['review_description'] = descrip_parsed

content_id_split1 = jeen_decoded.split('><meta name="id" content="')[1]
content_id_parsed = content_id_split1.split('"/><meta name="keywords" content="')[0]
#song_dict_reg[key_string]['content_id'] = content_id_parsed

photo_id_split1 = jeen_decoded.split(r'https://media.pitchfork.com/photos/')[1]
photo_id_parsed = photo_id_split1.split(r'/16:9/w_1000,c_limit/')[0]
#song_dict_reg[key_string]['photo_id'] = photo_id_parsed

full_photo_url = r'https://media.pitchfork.com/photos/' + photo_id_parsed + r'/master/w_1280,c_limit'
#song_dict_reg[key_string]['full_photo_url'] = full_photo_url

In [ ]:
waybackmachine_url = r'https://web.archive.org/web/20160310042721/http://pitchfork.com/reviews/albums/3/'

req = urllib.request.Request(waybackmachine_url)
r = urllib.request.urlopen(req).read()
peen_decoded = r.decode("utf-8")

peen_decoded